In [ ]:
import json
import os
import cv2
import glob
from tqdm.auto import tqdm
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from tqdm import tqdm
import shutil
from collections import defaultdict
import math

In [ ]:
# 일정 길이 이하 프레임 라벨 지우기
label_dir_path = '/workspace/Final_Submission/sorted_result/*.txt'
processed_label_dir_path = '/workspace/Final_Submission/post/post_0/'
label_paths = glob.glob(label_dir_path)
for path in tqdm(label_paths):
    res = []
    filename = os.path.basename(path)
    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(float,value_list))
        value_list = list(map(int,value_list))
        res.append(value_list)
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
    f.close()
    f = open(path, 'r')
    t = open(processed_label_dir_path + filename, 'w')
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        label = int(value_list[1])
        if(label_max_dict[label] - label_min_dict[label] > 20):
            t.write(line + '\n')
        else:
            if("THERMAL" in filename):
                t.write(line + '\n')
            continue
    t.close()
    f.close()

In [ ]:
# 합쳐졌다가 새로운 라벨 생기지않도록
processed_label_dir_path = '/workspace/Final_Submission/post/post_0/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_1/'
processed_label_dir_path = glob.glob(processed_label_dir_path)
for path in tqdm(processed_label_dir_path):
    filename = os.path.basename(path)
    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
    f.close()
    f = open(path, 'r')
    res = []
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        label = int(value_list[1])
        frame = int(value_list[0])
        candidate = -1
        for key in list(label_min_dict.keys()):
            if key == label:
                continue
            if (label_max_dict[key] < label_min_dict[label]) and (label_min_dict[key] < label_min_dict[label]):
                if candidate == -1:
                    candidate = key
                else:
                    if(label_min_dict[candidate] > label_min_dict[key]):
                        candidate = key
        if(candidate != -1):
            value_list[1] = candidate
            if(frame == label_max_dict[label]):
                        label_max_dict[candidate] = frame

        [fr,la,_,_,_,_,_,_,_,_] = value_list
        if len(list(filter(lambda x: x[0]==fr and x[1]==la,res))) != 0:
            continue
        res.append(value_list)
    f.close()
    
    res.sort(key=lambda x: int(x[1]))

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in res:
        l = list(map(str,l))
        t.write(','.join(l) + '\n')
    t.close()

In [ ]:
# 연속적으로 프레임 생성
import math

def length(x1,y1,x2,y2,height,width):
    x1 = x1/width
    x2 = x2/width
    y1 = y1/height
    y2 = y2/height
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

def fill_empty_frame(res,fr,label,before,height,width):
    threshold = 0.03

    _,_,x1,y1,a,b,c,d,e,f = before
    # 1. 주변 가까운(중심점 거리 threshold) frame으로 한다.
    candidate_frames = list(filter(lambda x: x[0]==fr,res))
    near = []
    for idx, frame in enumerate(candidate_frames):
        _,_,x2,y2,_,_,_,_,_,_ = frame
        distance = length(x1,y1,x2,y2,height,width)
        if distance <= threshold:
            near.append((idx,distance))
    if near != []:
        near.sort(key = lambda x: x[1])
        l = list(candidate_frames[near[0][0]])
        l[1] = label
        return l
    # 2. 과거의 frame으로 한다.
    else:
        return [fr,label,x1,y1,a,b,c,d,e,f]


processed_label_dir_path = '/workspace/Final_Submission/post/post_1/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_2/'
test_video_path = '/workspace/data/Test/'
processed_label_dir_path = glob.glob(processed_label_dir_path) 
for path in tqdm(processed_label_dir_path):
    res=[]
    filename = os.path.basename(path)

    videoname = filename.replace('.txt','')
    image = cv2.imread(test_video_path+videoname+'/img1/000001.jpg', cv2.IMREAD_COLOR)
    height, width, layers = image.shape

    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
        res.append(value_list)
    f.close()

    for key in label_max_dict.keys():
        before = list(filter(lambda x : x[0]==label_min_dict[key] and x[1]==key,res))[0]
        for fr in range(label_min_dict[key],label_max_dict[key]+1):
            now = list(filter(lambda x : x[0]==fr and x[1]==key,res))
            if len(now) == 0: # 해당 frame 비어있음
                frame = fill_empty_frame(res,fr,key,before,height,width)
                before = frame
                res.append(frame)
            else: # 해당 frame 존재
                before = now[0]

    sort_list = dict()
    final = []
    for key in label_max_dict.keys():
        label_sort_list = list(filter(lambda x : x[1]==key,res))
        label_sort_list.sort(key=lambda x: x[0])
        final += label_sort_list

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in final:
        l = list(map(str,l))
        t.write(','.join(l) + '\n')
    t.close()

In [ ]:
# 일정거리를 움직이지 않은 객체는 제거
import math

def length(x1,y1,x2,y2,height,width):
    x1 = x1/width
    x2 = x2/width
    y1 = y1/height
    y2 = y2/height
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

processed_label_dir_path = '/workspace/Final_Submission/post/post_2/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_3/'
test_video_path = '/workspace/data/Test/'
processed_label_dir_path = glob.glob(processed_label_dir_path)
for path in tqdm(processed_label_dir_path):
    res = []
    filename = os.path.basename(path)
    videoname = filename.replace('.txt','')
    image = cv2.imread(test_video_path+videoname+'/img1/000001.jpg', cv2.IMREAD_COLOR)
    height, width, layers = image.shape
    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
        res.append(value_list)
    f.close()

    except_list = []
    for key in label_max_dict.keys():
        fr_min = label_min_dict[key]
        fr_max = label_max_dict[key]
        _,_,x1,y1,_,_,_,_,_,_ = list(filter(lambda x : x[0]==fr_min and x[1]==key,res))[0]
        _,_,x2,y2,_,_,_,_,_,_ = list(filter(lambda x : x[0]==fr_max and x[1]==key,res))[0]
        if(length(x1,y1,x2,y2,height,width)<0.01):
           except_list.append(key) 

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in res:
        if l[1] not in except_list:
            l = list(map(str,l))
            t.write(','.join(l) + '\n')
    t.close()


In [ ]:
# 다른 객체에 비해 덜 존재하는 bbox는 객체에서 제거
processed_label_dir_path = '/workspace/Final_Submission/post/post_3/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_4/'
test_video_path = '/workspace/data/Test/'
processed_label_dir_path = glob.glob(processed_label_dir_path)
for path in tqdm(processed_label_dir_path):
    res = []
    filename = os.path.basename(path)
    videoname = filename.replace('.txt','')

    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
        res.append(value_list)
    f.close()

    if (len(label_max_dict.keys())==3):
        mean_frame = 0
        for key in label_max_dict.keys():
            fr_min = label_min_dict[key]
            fr_max = label_max_dict[key]
            mean_frame += fr_max - fr_min
        
        mean_frame /= len(label_max_dict.keys())
        except_key = []
        for key in label_max_dict.keys():
            fr_min = label_min_dict[key]
            fr_max = label_max_dict[key]
            if fr_max-fr_min < mean_frame:
                except_key.append(key)

        res = list(filter(lambda x: x[1] not in except_key, res))

    sort_list = dict()
    final = []
    for key in label_max_dict.keys():
        label_sort_list = list(filter(lambda x : x[1]==key,res))
        label_sort_list.sort(key=lambda x: x[0])
        final += label_sort_list

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in final:
        l = list(map(str,l))
        t.write(','.join(l) + '\n')
    t.close()




In [ ]:
# 처음에 합쳐져 시작하기 120fr이상에서 중간에 나타날경우
processed_label_dir_path = '/workspace/Final_Submission/post/post_4/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_5/'
test_video_path = '/workspace/data/Test/'
processed_label_dir_path = glob.glob(processed_label_dir_path)
for path in tqdm(processed_label_dir_path):
    res = []
    filename = os.path.basename(path)
    videoname = filename.replace('.txt','')

    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
        res.append(value_list)
    f.close()

    for key in label_max_dict.keys():
        fr_min = label_min_dict[key]
        if(fr_min > 120):
            for fr in range(1,fr_min):
                if len(list(filter(lambda x : x[0]==fr,res))) == 0:
                    break
                else:
                    [_,_,a,b,c,d,e,f,g,h] = list(filter(lambda x : x[0]==fr,res))[0]
                    res.append([fr,key,a,b,c,d,e,f,g,h])

    sort_list = dict()
    final = []
    for key in label_max_dict.keys():
        label_sort_list = list(filter(lambda x : x[1]==key,res))
        label_sort_list.sort(key=lambda x: x[0])
        final += label_sort_list

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in final:
        l = list(map(str,l))
        t.write(','.join(l) + '\n')
    t.close()


In [ ]:
# 중간에서 끝나는 프레임 계속 끝까지 유지하기 
processed_label_dir_path = '/workspace/Final_Submission/post/post_5/*.txt'
processed_label_dir_path2 = '/workspace/Final_Submission/post/post_6/'
test_video_path = '/workspace/data/Test/'
processed_label_dir_path = glob.glob(processed_label_dir_path)
for path in tqdm(processed_label_dir_path):
    res = []
    filename = os.path.basename(path)
    print(filename)
    videoname = filename.replace('.txt','')

    f = open(path, 'r')
    label_min_dict = defaultdict(int)
    label_max_dict = defaultdict(int)
    for line in f.readlines():
        line = line.strip()
        value_list = line.split(',')
        value_list = list(map(int,value_list))
        frame = int(value_list[0])
        label = int(value_list[1])
        if(label_min_dict[label] == 0):
            label_min_dict[label] = frame
        if(label_max_dict[label] <= frame):
            label_max_dict[label] = frame
        res.append(value_list)
    f.close()
    
    videoname = filename.replace('.txt','')
    image = cv2.imread(test_video_path+videoname+'/img1/000001.jpg', cv2.IMREAD_COLOR)
    height, width, layers = image.shape
    max_frame = res[-1][0]
    for key in label_max_dict.keys():
        last_frame = list(filter(lambda x : x[1]==key and x[0]==label_max_dict[key],res))[0]
        _,_,left,top,a,b,c,d,e,f = last_frame
        l = left/width
        t = top/height
        if not(l<0.05 or l>0.95 or t>0.95 or t<0.05):
            if('THERM' in filename):
                for fr in range(last_frame[0]+1,max_frame+1):
                    print('append!! : ', [fr,key,left,top,a,b,c,d,e,f])
                    res.append([fr,key,left,top,a,b,c,d,e,f])

    for key in label_max_dict.keys():
        label_sort_list = list(filter(lambda x : x[1]==key,res))
        label_sort_list.sort(key=lambda x: x[0])
        final += label_sort_list

    t = open(processed_label_dir_path2 + filename, 'w')
    for l in final:
        l = list(map(str,l))
        t.write(','.join(l) + '\n')
    t.close()